# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, ρin::RealFourierArray, ρout::RealFourierArray; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing
        ρin + mixing.α * (ρout - ρin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, ρin, ρout; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -7.227950691222         NaN   3.22e-01    0.0 
  2   -7.245804422997   -1.79e-02   1.48e-01    0.0 
  3   -7.248703291891   -2.90e-03   6.68e-02    0.0 
  4   -7.249049862961   -3.47e-04   3.67e-02    0.0 
  5   -7.249162347220   -1.12e-04   2.03e-02    0.0 
  6   -7.249198751658   -3.64e-05   1.14e-02    0.0 
  7   -7.249210699448   -1.19e-05   6.45e-03    0.0 
  8   -7.249214714497   -4.02e-06   3.71e-03    0.0 
  9   -7.249216103408   -1.39e-06   2.16e-03    0.0 
 10   -7.249216598954   -4.96e-07   1.28e-03    0.0 
 11   -7.249216781178   -1.82e-07   7.67e-04    0.0 
 12   -7.249216850059   -6.89e-08   4.67e-04    0.0 
 13   -7.249216876726   -2.67e-08   2.88e-04    0.0 
 14   -7.249216887257   -1.05e-08   1.79e-04    0.0 
 15   -7.249216891483   -4.23e-09   1.13e-04    0.0 


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.